In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0

In [3]:
N_centr = 10
eIDbins = [2]
primaryID = [[22,23,24,25],[23,24,25],[24,25],[25]]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
real_file_path = 'input/'
real_file_names=['my-m_ee_Run14AuAu_55th_new_19657_1072runs.root'] # m_ee_Run14AuAu_55th_new_19657_1072runs m_ee_Run14AuAu_39th_new_19579_900runs m_ee_Run14AuAu_54th_new_19654_1072runs
sim_file_path="../sim/output/Newembed/sngl/"
primary_file_names=['phi_50M_v7.root']#['ccbar_50M_v0.root']
hadron_file_names=['charged_pi_100M_v3.root']
dalitz_file_names=['dalitz_50M_v7.root']
conv_file_names=['photon_300M_v1.root']
sim_file_names = [primary_file_names,dalitz_file_names,conv_file_names,hadron_file_names ]
part_names = ["c#bar{c}","dalitz","#gamma #rightarrow ee","#pi^{-}"]
hist_names = ["el_pt_hist_"]

In [4]:
hists_read_real, hists_read_sim = [], []

for iFilereal in range(len(real_file_names)):
    infile = root.TFile.Open(real_file_path+real_file_names[iFilereal], "read")
    hists_read_real_ifile = []
    for icentr in range(N_centr):
        hists_read_real_ifile.append(infile.Get(hist_names[0]+f'{icentr}'))
        hists_read_real_ifile[-1].SetDirectory(root.nullptr)
        hists_read_real_ifile[-1].SetName(hist_names[0]+f'{icentr}'+'_real')
    hists_read_real.append(hists_read_real_ifile)
    infile.Close()

for iFilesim in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFilesim][0], "read")
    hists_read_sim_ifile = []
    for icentr in range(N_centr):
        hists_read_sim_ifile.append(infile.Get(hist_names[0]+f'{icentr}'))
        hists_read_sim_ifile[-1].SetDirectory(root.nullptr)
        hists_read_sim_ifile[-1].SetName(hist_names[0]+f'{icentr}'+f'_sim{iFilesim}_')
    hists_read_sim.append(hists_read_sim_ifile)
    infile.Close()

In [5]:
hists_pt_real, hists_pt_sim = [], []

for iFilereal in range(len(real_file_names)):
    hists_pt_real_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_real_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_real_centr = []
            for icentr in range(N_centr):
                hists_pt_real_centr.append(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_real_centr[-1].Add(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_real_conv.append(hists_pt_real_centr)
        hists_pt_real_eid.append(hists_pt_real_conv)
    hists_pt_real.append(hists_pt_real_eid)
    

for iFilesim in range(len(sim_file_names)):
    hists_pt_sim_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_sim_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_sim_centr = []
            for icentr in range(N_centr):
                hists_pt_sim_centr.append(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_sim_centr[-1].Add(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_sim_conv.append(hists_pt_sim_centr)
        hists_pt_sim_eid.append(hists_pt_sim_conv)
    hists_pt_sim.append(hists_pt_sim_eid)

In [6]:
if False:
    c0 = root.TCanvas(f"c0",f"c0",1400,1400)
    c0.Divide(2,3)
    for icentr in range(N_centr):
        c0.cd(icentr+1)
        root.gPad.SetLogy()
        hists_pt_sim[1][0][2][icentr].Draw()
    c0.Draw()

signal = primary + conv + dalitz => 1 = (1 - x - y) + x + y

applying cuts

fraction_signal0 = fraction_primary0 * (1 - x - y) + fraction_conv0 * x + fraction_dalitz0 * y

fraction_signal1 = fraction_primary1 * (1 - x - y) + fraction_conv1 * x + fraction_dalitz1 * y

x = [ s1 + a1 * (s0-c0) / (c0-a0) - c1 * (s0-a0)/(c0-a0) ] / [ b1 + a1 * (b0-c0) / (c0-a0) - c1 * (b0-a0)/(c0-a0) ]

y = [ (s0-a0) - (b0-a0) * x ] / (c0 - a0)

In [7]:
rebin = 1
for icentr in range(N_centr):
    for ieID in range(0,len(eIDbins)):
        for iconv in range(len(primaryID)):
            hists_pt_real[0][ieID][iconv][icentr].Rebin(rebin)
            hists_pt_sim[0][ieID][iconv][icentr].Rebin(rebin)
            hists_pt_sim[1][ieID][iconv][icentr].Rebin(rebin)

In [8]:
graphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)):
        graphs_eid = []
        for iconv in range(1,3):
            graphs_conv = []
            for iptbin in range(1,1+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                s0 = hists_pt_real[0][ieID][1][icentr].GetBinContent(iptbin)/(hists_pt_real[0][ieID][0][icentr].GetBinContent(iptbin)+1e-6)
                a0 =  hists_pt_sim[0][ieID][1][icentr].GetBinContent(iptbin)/(hists_pt_sim[0][ieID][0][icentr].GetBinContent(iptbin)+1e-6)
                b0 =  hists_pt_sim[1][ieID][1][icentr].GetBinContent(iptbin)/(hists_pt_sim[1][ieID][0][icentr].GetBinContent(iptbin)+1e-6)
                c0 =  hists_pt_sim[2][ieID][1][icentr].GetBinContent(iptbin)/(hists_pt_sim[2][ieID][0][icentr].GetBinContent(iptbin)+1e-6)
                s1 = hists_pt_real[0][ieID][1+iconv][icentr].GetBinContent(iptbin)/(hists_pt_real[0][ieID][0][icentr].GetBinContent(iptbin)+1e-6)
                a1 =  hists_pt_sim[0][ieID][1+iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[0][ieID][0][icentr].GetBinContent(iptbin)+1e-6)
                b1 =  hists_pt_sim[1][ieID][1+iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[1][ieID][0][icentr].GetBinContent(iptbin)+1e-6)
                c1 =  hists_pt_sim[2][ieID][1+iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[2][ieID][0][icentr].GetBinContent(iptbin)+1e-6)
                if (c0 - a0) == 0 or b1 + a1 * (b0-c0) / (c0-a0) - c1 * (b0-a0)/(c0-a0) == 0 : x,y = 0,0
                else :
                    x = ( s1 + a1 * (s0-c0) / (c0-a0) - c1 * (s0-a0)/(c0-a0) ) / ( b1 + a1 * (b0-c0) / (c0-a0) - c1 * (b0-a0)/(c0-a0) )
                    y = ( (s0-a0) - (b0-a0) * x ) / (c0 - a0)
                if x<0: x = 0
                if x>1: x = 2
                if y<0: 
                    y=0 
                x_err = x * (hists_pt_sim[2][ieID][1+iconv][icentr].GetBinError(iptbin)/(hists_pt_sim[2][ieID][1+iconv][icentr].GetBinContent(iptbin)+1e-6))*2**0.5
                y_err = y * (hists_pt_sim[2][ieID][1+iconv][icentr].GetBinError(iptbin)/(hists_pt_sim[2][ieID][1+iconv][icentr].GetBinContent(iptbin)+1e-6))*2**0.5
                graphs_conv.append([hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin),x,x_err,y,y_err])
                #print(s0, a0*(1-x-y)+b0*x+c0*y, s1, a1*(1-x-y)+b1*x+c1*y)
                #print(x,y,s0,a0,b0,c0,s1,a1,b1,c1)
                #print(x,y,s0 - (a0*(1-x-y)+b0*x+c0*y), s1 - (a1*(1-x-y)+b1*x+c1*y))
            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    graphs.append(graphs_centr)
print(graphs)

[[[[[0.05, 0, 0.0, 0, 0.0], [0.15000000000000002, 2, 2.828424296321894, 0, 0.0], [0.25, 0, 0.0, 46.876317134256304, 6.874276596647766], [0.35000000000000003, 2, 0.22941573236124774, 0, 0.0], [0.45, 0.5806055903083847, 0.053677005582865416, 0, 0.0], [0.55, 0.5571985407383373, 0.05324795200174182, 0, 0.0], [0.6500000000000001, 0.2741087570822402, 0.028049247999254234, 0.028877206366186313, 0.0029549728053663953], [0.7500000000000001, 0.653097802631276, 0.05706141973774391, 0, 0.0], [0.8500000000000001, 2, 0.19289712797096892, 0, 0.0], [0.9500000000000001, 0.4280850743756257, 0.03983264006894181, 0.06707402270347695, 0.0062411318783315], [1.05, 0.8083576134853779, 0.07833010832303443, 0, 0.0], [1.1500000000000001, 0.367179949817332, 0.03662654261521048, 0.10005854728300499, 0.009980933457560067], [1.2500000000000002, 0, 0.0, 0.31936494515731484, 0.03139186134254579], [1.35, 0.5454883323813278, 0.058651268516082045, 0, 0.0], [1.4500000000000002, 0, 0.0, 0.37904455728986514, 0.0408734255425

In [9]:
Tgraphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(len(eIDbins)):
        graphs_eid = []
        for iconv in range(2):
            graphs_conv = root.TGraphErrors()
            ipoint = 0
            for iptbin in range(0,0+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                graphs_conv.AddPoint(hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1),graphs[icentr][ieID][iconv][iptbin][1])
                graphs_conv.SetPointError(ipoint,hists_pt_real[0][ieID][iconv][icentr].GetBinWidth(iptbin+1)/2,graphs[icentr][ieID][iconv][iptbin][2])
                ipoint+=1

            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    Tgraphs.append(graphs_centr)

In [10]:
Dgraphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(len(eIDbins)):
        graphs_eid = []
        for iconv in range(2):
            graphs_conv = root.TGraphErrors()
            ipoint = 0
            for iptbin in range(0,0+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                graphs_conv.AddPoint(hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1),graphs[icentr][ieID][iconv][iptbin][3])
                graphs_conv.SetPointError(ipoint,hists_pt_real[0][ieID][iconv][icentr].GetBinWidth(iptbin+1)/2,graphs[icentr][ieID][iconv][iptbin][4])
                ipoint+=1

            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    Dgraphs.append(graphs_centr)

In [11]:
c0 = root.TCanvas(f"c0",f"c0",1400,1000)
c0.Divide(3,3)
h = []
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)):
        graphs_eid = []
        c0.cd(icentr+1)
        h.append(root.TH1D(f"name{icentr}{ieID}{0}", f"name{icentr}{ieID}{0}", 100, 0.1, 4.9))
        h[-1].SetMaximum(1.1)
        Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        h[-1].Draw()
        Format_Graph( Tgraphs[icentr][ieID][0],  Mstyle=21,  Msize=1,  Mcolor=1,  Lwidth=1,  Lcolor=1,  Malpha=1,  Lalpha=1)
        Tgraphs[icentr][ieID][0].Draw("same P")
        Format_Graph( Tgraphs[icentr][ieID][1],  Mstyle=20,  Msize=1,  Mcolor=2,  Lwidth=1,  Lcolor=1,  Malpha=2,  Lalpha=1)
        Tgraphs[icentr][ieID][1].Draw("same P")
        #Tgraphs[icentr][ieID][1].Fit("pol0","w","w")
c0.Draw()

In [12]:
c0 = root.TCanvas(f"c0",f"c0",1400,1000)
c0.Divide(3,3)
h = []
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)):
        graphs_eid = []
        c0.cd(icentr+1)
        h.append(root.TH1D(f"name{icentr}{ieID}{0}", f"name{icentr}{ieID}{0}", 100, 0.1, 4.9))
        h[-1].SetMaximum(1.1)
        Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        h[-1].Draw()
        Format_Graph( Dgraphs[icentr][ieID][0],  Mstyle=21,  Msize=1,  Mcolor=1,  Lwidth=1,  Lcolor=1,  Malpha=1,  Lalpha=1)
        Dgraphs[icentr][ieID][0].Draw("same P")
        Format_Graph( Dgraphs[icentr][ieID][1],  Mstyle=20,  Msize=1,  Mcolor=2,  Lwidth=1,  Lcolor=1,  Malpha=2,  Lalpha=1)
        Dgraphs[icentr][ieID][1].Draw("same P")
c0.Draw()

In [13]:
if True :
    only_pos = 1
    eID = 0
    c0 = root.TCanvas(f"c0",f"c0",1400,1100*(2-only_pos))
    N_centr_loc = N_centr - 5*only_pos
    c0.Divide(3,2*(2-only_pos))
    for icentr in range(N_centr_loc):
        for iconv in range(1):
            c0.cd(icentr+1+N_centr_loc*iconv)
            root.gPad.SetLogy()
                
            Format_Hist_total(hists_pt_real[0][eID][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=2, Mcolor=2,  Lwidth=4,  Lcolor=2,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            
            hists_pt_real[0][eID][iconv][icentr].GetXaxis().SetRange(4,40)
            hists_pt_real[0][eID][iconv][icentr].Draw()
    
            for iptbin in range(1,1+hists_pt_real[0][0][iconv][icentr].GetNbinsX()):
                hists_pt_sim[1][0][iconv][icentr].SetBinContent(iptbin,0.8*hists_pt_real[0][eID][iconv][icentr].GetBinContent(iptbin))
                #hists_pt_sim[1][1][iconv][icentr].SetBinContent(iptbin,Tgraphs[icentr][3][1].GetFunction("pol2").Eval(hists_pt_real[0][0][iconv][icentr].GetBinCenter(iptbin))*hists_pt_real[0][0][iconv][icentr].GetBinContent(iptbin))
                
            Format_Hist_total(hists_pt_sim[1][0][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=4,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            
            hists_pt_sim[1][0][iconv][icentr].Draw('same H')
            #hists_pt_sim[1][1][iconv][icentr].Draw('same H')
    c0.Draw()            

In [ ]:
if False:
    outfilename = f"output/files/dalitz.root"
    outfile = root.TFile.Open(outfilename, "recreate")
    for icentr in range(N_centr_loc):
        hists_pt_real[0][0][0][icentr].Write()
        hists_pt_sim[1][0][0][icentr].Write()
    outfile.Close()